<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/2_2%EB%B2%88_In_Ability_%EC%A0%95%EA%B7%9C%EC%8B%9D%ED%99%9C%EC%9A%A9_Generating_Features_Time_to12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')

#%time
#test = pd.read_pickle('./test.pkl')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


## Train 으로부터 mytrain 데이터 생성

In [0]:
Ability = train.loc[train['event']=='Ability',:]
import re
def SUB(x):
    result = re.sub(' ','',str(x)) + ','
    #result = re.sub('[[0-9]+]',    '',     result)
    result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
    #result = re.sub('Location:.+',    'Location:',     result)
    return result

Ability['event_contents'] = Ability.apply(lambda x: SUB(x['event_contents']), axis=1)
Ability['event_contents'].value_counts()

()-Attack;Location:(),                                                      457150
()-TrainSCV,                                                                350376
()-MorphDrone,                                                              287254
()-TrainProbe,                                                              258765
()-TrainMarine,                                                             134801
                                                                             ...  
()-ScanMove;Target:CreepTumorQueen[];Location:(),                                1
()-Attack;Target:NydusNetwork[];Location:(),                                     1
()-BuildAssimilator;Target:AiurLightBridgeAbandonedNE8Out[];Location:(),         1
()-ScanMove;Target:SporeCrawler[];Location:(),                                   1
()-MULERepair;Target:LocustMPFlying[];Location:(),                               1
Name: event_contents, Length: 1232, dtype: int64

In [0]:
game_id = train.game_id.unique().tolist()
columns = ['text0','text1','winner']
mytrain = np.zeros(len(game_id)*len(columns))
mytrain = mytrain.reshape(len(game_id),len(columns))
mytrain = pd.DataFrame(mytrain, columns=columns)
mytrain.index = game_id
mytrain.winner = train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
mytrain.loc[:,'text0'] = Ability.loc[Ability['player']==0,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
mytrain.loc[:,'text1'] = Ability.loc[Ability['player']==1,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
del train
del Ability
mytrain = mytrain.fillna('')


## Test 로부터 mytest 데이터 생성

In [0]:
test = pd.read_csv('test.csv')
Ability = test.loc[test['event']=='Ability',:]
import re
def SUB(x):
    result = re.sub(' ','',str(x)) + ','
    #result = re.sub('[[0-9]+]',    '',     result)
    result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
    #result = re.sub('Location:.+',    'Location:',     result)
    return result

Ability['event_contents'] = Ability.apply(lambda x: SUB(x['event_contents']), axis=1)
Ability['event_contents'].value_counts()

3                       ()-TrainProbe,
29                      ()-TrainProbe,
69          ()-BuildPylon;Location:(),
75                      ()-TrainProbe,
82                      ()-TrainProbe,
                       ...            
28714832        ()-Attack;Location:(),
28714834        ()-Attack;Location:(),
28714836        ()-Attack;Location:(),
28714837        ()-Attack;Location:(),
28714839        ()-Attack;Location:(),
Name: event_contents, Length: 1631055, dtype: object

In [0]:
game_id = test.game_id.unique().tolist()
columns = ['text0','text1']
mytest = np.zeros(len(game_id)*len(columns))
mytest = mytest.reshape(len(game_id),len(columns))
mytest = pd.DataFrame(mytest, columns=columns)
mytest.index = game_id

mytest.loc[:,'text0'] = Ability.loc[Ability['player']==0,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
mytest.loc[:,'text1'] = Ability.loc[Ability['player']==1,:].loc[Ability['event']=='Ability',:].groupby(['game_id']).event_contents.sum()
del test
del Ability
mytest = mytest.fillna('')


## Countvectorizer & Tfidf 연습 (모델링말고)

In [0]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

def tokenizer(text):
    return text.split(',')

In [0]:
vectorizer = CountVectorizer(min_df=2, ngram_range=(1,2))
revised_reviews = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews = vectorizer.fit_transform(revised_reviews)
print(vector_reviews.shape)
print(vectorizer.get_feature_names())

(38872, 8451)
['adept', 'adept location', 'adeptphaseshift', 'adeptphaseshift location', 'adeptphaseshiftcancel', 'adeptphaseshiftcancel adeptphaseshift', 'adeptphaseshiftcancel attack', 'adeptphaseshiftcancel buildassimilator', 'adeptphaseshiftcancel buildforge', 'adeptphaseshiftcancel buildgateway', 'adeptphaseshiftcancel buildnexus', 'adeptphaseshiftcancel buildpylon', 'adeptphaseshiftcancel buildroboticsfacility', 'adeptphaseshiftcancel chronoboost', 'adeptphaseshiftcancel holdposition', 'adeptphaseshiftcancel oracleweapon', 'adeptphaseshiftcancel patrol', 'adeptphaseshiftcancel phasingmode', 'adeptphaseshiftcancel stop', 'adeptphaseshiftcancel trainadept', 'adeptphaseshiftcancel trainimmortal', 'adeptphaseshiftcancel trainmothershipcore', 'adeptphaseshiftcancel trainobserver', 'adeptphaseshiftcancel trainoracle', 'adeptphaseshiftcancel trainphoenix', 'adeptphaseshiftcancel trainprobe', 'adeptphaseshiftcancel trainstalker', 'adeptphaseshiftcancel trainvoidray', 'adeptphaseshiftcanc

In [0]:
tfidf = TfidfTransformer()
np.set_printoptions(precision=4)    ## print하는 데이터의 유효숫자를
print(tfidf.fit_transform(vector_reviews).toarray())

[[0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.0359 ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 ...
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]]


## 파이프라인 : TFIDF & Naive Bayes

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
x = mytrain.loc[:, 'text0'].values
y = mytrain.loc[:, 'winner'].values
X_train, X_valid, y_train, y_valid = train_test_split(x, y,   stratify=y, random_state=42)


#X_train = mytrain.loc[:, 'text'].values
#y_train = mytrain.loc[:, 'winner'].values
def tokenizer(text):
    return text.split(',')

vectorizer = CountVectorizer(min_df=1)
tfidf = TfidfVectorizer(tokenizer=tokenizer,min_df=1)
multi_nbc = Pipeline([('vect', tfidf), ('nbc', MultinomialNB())])


In [0]:
multi_nbc.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer at 0x7fa953a847b8>,
                                 use_idf=True, vocabulary=None)),
                ('nbc',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
y_pred = multi_nbc.predict(X_valid)
print("테스트 정확도: {:.4f}".format(accuracy_score(y_valid, y_pred)))


테스트 정확도: 0.536


## PipeLine 없이 모델링

### 1. CountVevtorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 1
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
del player0, player1


(38872, 504)
(38872, 500)
(38872, 1004)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
min_df = 1
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

del player0, player1

(16787, 476)
(16787, 475)
(16787, 951)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

1004
951
936


## 단어임베딩 저장한 pkl 불러와서 Training

In [0]:

###################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=seed).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포 #  ngram (1,1) : 0.6883 vs ngram(1,2) : 
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM #  ngram (1,1) : 0.6824 vs ngram(1,2) : 
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.6883
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000,

## (예측을 위함)테스트 데이터 단어가방 생성

In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
min_df = 1
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

del player0, player1

(16787, 476)
(16787, 475)
(16787, 951)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

need_del_col = list(set(X_test.columns) - set(X.columns))
X_test = X_test.drop(need_del_col, axis=1)  # Test에만 있는 컬럼 제거

need_add_col = list(set(X.columns) - set(X_test.columns))
need_add = np.zeros(X_test.shape[0]*len(need_add_col))
need_add = need_add.reshape(X_test.shape[0], len(need_add_col))
need_add = pd.DataFrame(need_add, columns = need_add_col)
X_test = pd.concat([X_test,need_add],axis=1)
print(len(X_test.columns))


1004
951
936
1004


In [0]:
X.to_pickle('train_Ability12.pkl')
X_test.to_pickle('test_Ability12.pkl')